## Core Functions
- Report the range of times for each event for the given data set (done)
- Report the median time for each for the selected data set (done)
- Report fastest for each event along with that person and the rest of their time (EX "Christina Hepper has the fastest Swim at _____ and had the following other times...., Jamie Fend had the fastest T1 at _____ and had teh following other times...." (done, but need to improve formatting of result)
- Create functions
-- function to convert to integer minutes (done)
-- function for performance standardizing (done)
- Compute the fastest theoretical time (done)
- Compute the median theoretical time
- Report how selected Bib Number did relative to lead (EX "989 was 0.9x at swimming, 1.1x at T1....")
- Create a line plot with each event on the X, the Standardized Time on the Y (standardize to fastest?) and each line being an athlete, highlight the user and the median line
- Need to cumsum the colums

## Future
- Use mongo db to store data
- Automatically scrape the results
- Compute a "competitiveness" score for each race (like FSAE world: https://fs-world.org/)
- Calculate the percentile for eahc event for the user

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
# from bokeh.plotting import figure, show
# from sqlalchemy import create_engine
# pd.set_option("precision", 2)

In [ ]:
data = pd.read_excel('santa_cruz_data.xlsx', header = 0, index_col=None)
data.head()

#testdf = pd.read_html("https://results.svetiming.com/Santa-Cruz-Triathlon/events/2021/Santa-Cruz-Triathlon/results", attrs={'class': 'table table-condensed table-bordered table-striped table-hover'})

We need to convert the DataTime objects contatined in the data frame to integer minutes to do all of our nice math. Use the function and create new columsn to hold the data.

Create a second set of parameters for cumulative times to use later

In [ ]:
actdf = testdf[0]
type(actdf)

actdf

In [ ]:
#create a function to convert the time to integer seconds
#To create a new column, use the [] brackets with the new column name at the left side of the assignment.

def convertTime (time):
    timeMinutes = (time.hour*60)+(time.minute)+(time.second/60)
    return timeMinutes

#convert to integers
data["Swim Minutes"] = data["Swim"].apply(convertTime)
data["T1 Minutes"] = data["T1"].apply(convertTime)
data["Bike Minutes"] = data["Bike"].apply(convertTime)
data["T2 Minutes"] = data["T2"].apply(convertTime)
data["Run Minutes"] = data["Run"].apply(convertTime)
#data["Elapsed Minutes"] = data["Chip Elapsed"].apply(convertTime)

#create cumulative times
data["Swim+T1"]=data["Swim Minutes"]+data["T1 Minutes"]
data["Plus Bike"]=data["Swim+T1"]+data["Bike Minutes"]
data["Plus T2"]=data["Plus Bike"]+data["T2 Minutes"]
data["Total"]=data["Plus T2"]+data["Run Minutes"]

Get statistical data about the event.

In [ ]:
summary = data.describe()
data[["Swim Minutes", "T1 Minutes", "Bike Minutes", "T2 Minutes", "Run Minutes"]].describe()

In [ ]:
#assign user variable
user = "SAMANTHA DEWEES"
index = data.index[data['Name']==user].tolist()
data.loc[index, ["Name", "Age", "Gender Place", "Swim", "T1", "Bike", "T2", "Run"]]

In [ ]:
def getMinTime (stage):
    minTime = data[stage].min()
    minIndex = data[stage].idxmin()
    #print("minTime is", minTime)
    #print("minIndex is", minIndex)
    return minTime, minIndex
    
def printResults (minIndex):
    #need to make this so it doesnt print reduntant colums or prints something nicely formatted
    print(data.iloc[minIndex,:])

In [ ]:
events = ['Swim Minutes', 'T1 Minutes', 'Bike Minutes', 'T2 Minutes', 'Run Minutes']

shortestTime = 0
for stage in events:
    minTime, minIndex = getMinTime(stage)
    print("the fastest for ", stage)
    printResults(minIndex)
    shortestTime = shortestTime + minTime

print("the shortest possible time is: ", shortestTime)
#need to work on formatting

In [ ]:
reduced2 = data[["Name","Swim Minutes","Swim+T1","Plus Bike","Plus T2","Total","Gender Place"]]
reduced2["Start"] = 0
reduced2 = reduced2.drop(25)
reduced2

# color_nums = reduced2['Gender Place'].tolist()
# color_names = reduced2['Name'].tolist()
# colors_zip = zip(color_nums, color_names)

# colors_zip

In [ ]:
dimensions = list([
            dict(range = [0, 1],
                label = 'Start', values = reduced2['Start']),            
            dict(range = [reduced2["Swim Minutes"].min(), reduced2["Swim Minutes"].max()],
                label = 'Time After Swim', values = reduced2['Swim Minutes']),
            dict(range = [reduced2["Swim+T1"].min(), reduced2["Swim+T1"].max()],
                label = 'Time After First Transition', values = reduced2['Swim+T1']),
            dict(range = [reduced2["Plus Bike"].min(), reduced2["Plus Bike"].max()],
                label = 'Time After Bike', values = reduced2['Plus Bike']),
            dict(range = [reduced2["Plus T2"].min(), reduced2["Plus T2"].max()],
                label = 'Time After Second Transition', values = reduced2['Plus T2']),
            dict(range = [reduced2["Total"].min(), reduced2["Total"].max()],
                label = 'Total Time', values = reduced2['Total']),
            dict(range=[0,reduced2['Gender Place'].max()], tickvals = reduced2['Gender Place'], ticktext = reduced2['Name'],
                       label='Competitor', values=reduced2['Gender Place'])
        ])

fig = go.Figure(data=go.Parcoords(line = dict(color = reduced2['Gender Place'],
                   colorscale = [[.0,'rgba(255,0,0,0.1)'],[0.2,'rgba(0,255,0,0.1)'],[.4,'rgba(0,0,255,0.1)'], 
                                 [.6,'rgba(0,255,255,0.1)'], [.8, 'rgba(255,0,255,0.1)'], [1, 'rgba(0,0,0,0.1)']]), dimensions=dimensions))

fig.update_layout(
    title="Triathalon Results",
    width=1920,
    height=1080
)

fig.show()

In [ ]:
https://stackoverflow.com/questions/64139316/plotly-how-to-insert-a-categorical-variable-into-a-parallel-coordinates-plot

                   colorscale = [[.0,'rgba(218,79,79,0.1)'],[0.25,'rgba(148,218,79,0.1)'],[.5,'rgba(79,218,218,0.1)'], 
                                 [.75,'rgba(148,79,218,0.1)'], [1, 'rgba(0,0,0,0.1)']]), dimensions=dimensions))

In [ ]:
engine = create_engine('sqlite:///data.db', echo=False)

In [ ]:
sql_df = pd.read_sql('data.db', con=engine, index_col='ID')

In [ ]:
sql_df.head()

In [ ]:
scatter = px.scatter(x=data['Age'], y=data['Gender Place'])
scatter.show()
